In [1]:
!pip install mqttools

  Preparing metadata (setup.py) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for mqttools: filename=mqttools-0.50.0-py3-none-any.whl size=27301 sha256=10224272b901f0bbc077aa3233ef513a5149bc09fa467c91942dde5368a95e5c
  Stored in directory: /Users/josepsmachine/Library/Caches/pip/wheels/ba/d6/a5/0b3ec7272965496b529a68031b5086f981d09ec448ed149fdc
  Created wheel for bitstruct: filename=bitstruct-8.19.0-cp310-cp310-macosx_11_0_arm64.whl size=35979 sha256=e6a5e4009b416a2d4dba5619ebbb8c07d9509f768bd10f165380d516b7a8ce43
  Stored in directory: /Users/josepsmachine/Library/Caches/pip/wheels/63/5a/7b/4c174b451b9e05021d580be4ead7b3c6a8b5f82aac000ec371
Successfully built mqttools bitstruct


In [2]:
"""

 +--------+            +--------+            +-------------+
 |        |--- ping -->|        |--- ping -->|             |
 | client |            | broker |            | echo client |
 |        |<-- pong ---|        |<-- pong ---|             |
 +--------+            +--------+            +-------------+

"""

import asyncio
import time

import mqttools

BROKER_PORT = 10008


async def start_client():
    client = mqttools.Client('localhost', BROKER_PORT, connect_delays=[0.1])
    await client.start()

    return client


async def client_main():
    """Publish the current time to /ping and wait for the echo client to
    publish it back on /pong, with a one second interval.

    """

    client = await start_client()
    await client.subscribe('/pong')

    while True:
        print()
        message = str(int(time.time())).encode('ascii')
        print(f'client: Publishing {message} on /ping.')
        client.publish(mqttools.Message('/ping', message))
        message = await client.messages.get()
        print(f'client: Got {message.message} on {message.topic}.')

        if message is None:
            print('Client connection lost.')
            break

        await asyncio.sleep(1)


async def echo_client_main():
    """Wait for the client to publish to /ping, and publish /pong in
    response.

    """

    client = await start_client()
    await client.subscribe('/ping')

    while True:
        message = await client.messages.get()
        print(f'echo_client: Got {message.message} on {message.topic}.')

        if message is None:
            print('Echo client connection lost.')
            break

        print(f'echo_client: Publishing {message.message} on /pong.')
        client.publish(mqttools.Message('/pong', message.message))


async def broker_main():
    """The broker, serving both clients, forever.

    """

    broker = mqttools.Broker(('localhost', BROKER_PORT))
    await broker.serve_forever()


async def main():
    await asyncio.gather(
        broker_main(),
        echo_client_main(),
        client_main()
    )


asyncio.run(main())

RuntimeError: asyncio.run() cannot be called from a running event loop